In [3]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm 
import signal_processing.sr_processing as sr

In [4]:
dat_spike_paths = '/home/tony/Halluci-Nations/scratch/'
for main_path in os.listdir(dat_spike_paths):
    try:
        main_path = os.path.join(dat_spike_paths,main_path)
        time_res = 0.01
        bin_size = time_res
        data_path = os.path.join(main_path,'data.pkl')
        epoch_path = os.path.join(main_path,'epochs.pkl')
        photostim_path = os.path.join(main_path,'photostim.pkl')
        visualstim_path = os.path.join(main_path,'visualstim.pkl')
        df_data = pd.read_pickle(data_path)
        df_epoch = pd.read_pickle(epoch_path)
        df_photostim = pd.read_pickle(photostim_path)
        df_visualstim = pd.read_pickle(visualstim_path)
        stim_ep_list = [i for i in df_epoch['stim_name'] if 'RFMapping' in i]
        for stims_eps in stim_ep_list:
            epochs_path = f'{main_path}/{stims_eps}/'
            fr_path = f'{epochs_path}/firing_rate2/'
            trial_path = os.path.join(epochs_path,'trial_info.csv')
            df_trials = pd.read_csv(trial_path,index_col='Unnamed: 0')
            index_map = sr.get_index_maps(df_trials)
            results_dic ={}
            results_dic2 = {}
            pbar = tqdm(total = len(index_map.keys()),leave=True, position=0)
            for inds in index_map.keys():
                trials2avg = index_map[inds]
                data_set = {d_ind:sr.get_spikerate_data(f'{fr_path}trial_array_{d_ind}.pkl') for d_ind in trials2avg}
                if len(set([d['bin_centers'].shape[0] for d in data_set.values()])) != 1:
                    data_set = sr.trim_binsz_ts(data_set)
                temp_results = sr.psth_cal([trial['s_counts'] for trial in data_set.values()])/bin_size
                results_dic2[inds] = temp_results
                results_dic[inds] = sr.spike_rate_zscore(temp_results)
                pbar.update(1)
            filename = f'{epochs_path}/firing_rates_zscored.pkl'
            filename2 = f'{epochs_path}/firing_rates.pkl'
            pickle.dump(results_dic, open(filename, 'wb'))
            pickle.dump(results_dic2, open(filename2, 'wb'))
    except Exception as e:
        pass

 95%|█████████▌| 184/193 [00:01<00:00, 107.26it/s]

SystemExit: 

/home/tony/miniconda3/envs/brainsformer/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


100%|██████████| 193/193 [00:13<00:00, 107.26it/s]